## COVID 19 DATA CLEANING INDIA

Author: Rushabh Patel

Dashboard: 
1. https://covid19-dashboard-india.000webhostapp.com/index.html
2. https://public.tableau.com/views/COVID_FINAL/Home?:language=en&:display_count=y&publish=yes&:origin=viz_share_link

Data Sources:
1. https://api.covid19india.org/
2. https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv

Importing libraries

In [1]:
import numpy as np
import pandas as pd
import math
import datetime

Setting up working directory

In [2]:
path = 'C:/Users/rusha/Documents/COVID19/'

Country wise timeseries data

In [3]:
india_df = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')

#dropping & renaming columns
india_df = india_df.drop('Date',1)
india_df.rename(columns = {'Date_YMD' : 'Date'}, inplace=True)

india_df.to_excel(path+'india_df.xlsx', index=False)

District wise data

In [4]:
district_df = pd.read_csv('https://api.covid19india.org/csv/latest/district_wise.csv')

#dropping & renaming columns
district_df = district_df.drop(['SlNo'], 1)
district_df = district_df.drop([0], 0)

district_df.to_excel(path+'district_df.xlsx', index=False)

State wise timeseries data

In [5]:
state_daily_df = pd.read_csv('https://api.covid19india.org/csv/latest/states.csv')

#filling none values with 0 and manupilating the dataframe to bring it in the right format
state_daily_df = state_daily_df.fillna(0)
state_daily_df = state_daily_df.loc[state_daily_df['State']!='India',:].reset_index()
state_daily_df = state_daily_df.groupby(['State','Date']).sum()
state_daily_df = state_daily_df.reset_index()

#creating new columns
state_daily_df['Daily Confirmed'] = 0
state_daily_df['Daily Recovered'] = 0
state_daily_df['Daily Deceased'] = 0
state_daily_df['Daily Tested'] = 0

#calculating and assigning values to above created columns
for i in range(0,len(state_daily_df)):
    if(i>0 and state_daily_df.loc[i,'Confirmed'] >= 0):
        state_daily_df.loc[i,'Daily Confirmed'] = state_daily_df.loc[i,'Confirmed'] - state_daily_df.loc[i-1,'Confirmed']
        state_daily_df.loc[i,'Daily Recovered'] = state_daily_df.loc[i,'Recovered'] - state_daily_df.loc[i-1,'Recovered']
        state_daily_df.loc[i,'Daily Deceased'] = state_daily_df.loc[i,'Deceased'] - state_daily_df.loc[i-1,'Deceased']
        state_daily_df.loc[i,'Daily Tested'] = state_daily_df.loc[i,'Tested'] - state_daily_df.loc[i-1,'Tested']
    elif(i==0 and state_daily_df.loc[i,'Confirmed'] >= 0):
        state_daily_df.loc[i,'Daily Confirmed'] = state_daily_df.loc[i,'Confirmed']
        state_daily_df.loc[i,'Daily Recovered'] = state_daily_df.loc[i,'Recovered']
        state_daily_df.loc[i,'Daily Deceased'] = state_daily_df.loc[i,'Deceased']
        state_daily_df.loc[i,'Daily Tested'] = state_daily_df.loc[i,'Tested']
        

#deleting unnecessary column
del state_daily_df['index']


#forward filling values as per states
df_values = state_daily_df.values

for i in range(0,len(df_values)):
    state = df_values[i][0]
    daily_confirmed = df_values[i][7]
    daily_recovered = df_values[i][8]
    daily_deceased = df_values[i][9]
    daily_tested = df_values[i][10]
    
    if(daily_confirmed<0 and df_values[i-1][0]==df_values[i][0]):
        df_values[i][7] = state_daily_df.values[i-1][7]
    elif(daily_recovered<0 and df_values[i-1][0]==df_values[i][0]):
        df_values[i][8] = df_values[i-1][8]
    elif(daily_deceased<0 and df_values[i-1][0]==df_values[i][0]):
        df_values[i][9] = df_values[i-1][9]
    elif(daily_tested<0 and df_values[i-1][0]==df_values[i][0]):
        df_values[i][10] = df_values[i-1][10]


#creating final dataframe
state_daily_df = pd.DataFrame(df_values)

#manupilating and renaming columns with meaningful name
state_daily_df.columns = ['State', 'Date','Total Confirmed','Total Recovered','Total Deceased','Other','Total Tested',
                          'Daily Confirmed','Daily Recovered', 'Daily Deceased','Daily Tested']
state_daily_df['Date'] = state_daily_df['Date'].map(lambda x: x.replace('-','/'))

state_daily_df.to_excel(path+'state_daily_df.xlsx', index=False)

State wise testing data

In [22]:
state_test_df = pd.read_csv('https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv')

#preprocessing dataframe
state_test_df = state_test_df.ffill(axis = 0)
state_test_df = state_test_df.fillna(0)
state_test_df.rename(columns = {'Updated On' : 'Date'}, inplace=True)
state_test_df['Date'] = state_test_df['Date'].map(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y').strftime('%Y/%m/%d'))

#finding the average values of symptomatic and asymptomatic cases from all the states
avg_symptomatic_cases = int(np.mean(state_test_df['Cumulative Number of Symptomatic Cases']))
avg_asymptomatic_cases = int(np.mean(state_test_df['Cumulative Number of Asymptomatic Cases']))

#filling average values for the states with empty values/None in these columns
state_test_df['Cumulative Number of Symptomatic Cases'] = state_test_df['Cumulative Number of Symptomatic Cases'].map(lambda x: avg_symptomatic_cases if(x==0) else x)
state_test_df['Cumulative Number of Asymptomatic Cases'] = state_test_df['Cumulative Number of Asymptomatic Cases'].map(lambda x: avg_asymptomatic_cases if(x==0) else x)

state_test_df.to_excel(path+'state_test_df.xlsx',index=False)

State wise vaccine data

In [7]:
state_vaccine_df = pd.read_csv('http://api.covid19india.org/csv/latest/vaccine_doses_statewise.csv')

#melting the dataframe to bring in the right format
state_vaccine_df = pd.melt(state_vaccine_df, id_vars =['State'], value_vars = state_vaccine_df.columns[1:])

#renaming columns
state_vaccine_df.columns = ['State', 'Date', 'Total Vaccinated']

#bringing the dataframe in right format by matching the dates with other dataframe, filling empty values, 
#manupilating columns
states = state_vaccine_df['State'].unique()
state_vaccine_df = state_vaccine_df.sort_values(by=['State', 'Date'], ascending = True)
state_vaccine_df['Date'] = [datetime.datetime.strptime(i, '%d/%m/%Y').strftime('%m/%d/%Y') for i in state_vaccine_df['Date']]
state_vaccine_df = state_vaccine_df.groupby(by=['State','Date']).sum()
state_vaccine_df['Total Vaccinated'] = state_vaccine_df['Total Vaccinated'].astype(int).replace({0:None})
state_vaccine_df['Total Vaccinated'] = state_vaccine_df['Total Vaccinated'].ffill()
state_vaccine_df['Total Vaccinated'] = state_vaccine_df['Total Vaccinated'].fillna(0)
state_vaccine_df['Daily Vaccinated'] = 0

final_df = pd.DataFrame()

for s in states:
    temp_df = state_vaccine_df.loc[(s,),:]
    temp_df = temp_df.reset_index()
    values = temp_df.values
    start = values[0][0]
    end = values[-1][0]
    date_df = pd.DataFrame({'Date': pd.date_range(start=start, end=end)})
    temp_df['Date'] = [datetime.datetime.strptime(i, '%m/%d/%Y').strftime('%Y-%m-%d') for i in temp_df['Date']]
    date_df['Date'] = date_df['Date'].astype(str)
    temp_df['Date'] = temp_df['Date'].astype(str)
    temp_df = date_df.merge(temp_df, on='Date', how='left')
    temp_df['Total Vaccinated'] = temp_df['Total Vaccinated'].fillna(0).astype(int).replace({0:None})
    temp_df['Total Vaccinated'] = temp_df['Total Vaccinated'].ffill()
    temp_df['Total Vaccinated'] = temp_df['Total Vaccinated'].fillna(0)
    values = temp_df.values
    for i in range(0,len(temp_df)):
        if(i==0):
            values[i][2] = 0
        else:
            values[i][2] = values[i][1] - values[i-1][1]
    values = np.hstack((np.full(shape=(values.shape[0],1), fill_value=s),values))
    temp_df = pd.DataFrame(values)
    final_df = final_df.append(temp_df)

state_vaccine_df = final_df

#renaming columns and making each element absolute to avoid any typo error during data entry
state_vaccine_df.columns = ['State', 'Date', 'Total Vaccinated', 'Daily Vaccinated']
state_vaccine_df['Total Vaccinated'] = state_vaccine_df['Total Vaccinated'].map(lambda x: abs(x))
state_vaccine_df['Daily Vaccinated'] = state_vaccine_df['Daily Vaccinated'].map(lambda x: abs(x))
state_vaccine_df = state_vaccine_df.fillna(0)

#removing unnecessary rows associated with the following state values "Miscellaneous|Total"
state_vaccine_df = state_vaccine_df[~state_vaccine_df['State'].str.contains('Miscellaneous|Total')]

state_vaccine_df.to_excel(path+'state_vaccine_df.xlsx',index=False)

World wide confirmed cases data

In [8]:
world_df_time_c = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',error_bad_lines=False)

#dropping a column
world_df_time_c = world_df_time_c.drop(['Province/State'],1)

#melting the dataframe
world_df_time_c = pd.melt(world_df_time_c, id_vars =['Country/Region'], value_vars = world_df_time_c.columns[3:])

#renaming columns
world_df_time_c.columns = ['Country','Date','Confirmed']

#sorting by date and confirmed cases
world_df_time_c = world_df_time_c.sort_values(by=['Date','Confirmed'], ascending=False).reset_index()

#dropping columns
world_df_time_c = world_df_time_c.drop(['index','Date'],1)

#creating new column for the rank based on confirmed cases
world_df_time_c['Rank'] = 0

#includ rows till we find india in the country column of the world dataframe and discard the remaining rows
df_values = []
for i in range(0,len(world_df_time_c.values)):
    if(world_df_time_c.values[i][0]=='India'):
        df_values.append([world_df_time_c.values[i][0],world_df_time_c.values[i][1],i+1])
        break
    else:
        df_values.append([world_df_time_c.values[i][0],world_df_time_c.values[i][1],i+1])
        

#creating final dataframe
world_df_time_c = pd.DataFrame(df_values)

#renaming columns
world_df_time_c.columns = ['Country', 'Confirmed', 'Rank']

world_df_time_c.to_excel(path+'world_df_confirmed.xlsx',index=False)

Country wise testing data

In [9]:
india_test_df = pd.read_csv('https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv')

#filling empty values and dropping some columns
india_test_df = india_test_df.ffill(axis = 0)
india_test_df = india_test_df.drop(['Source', 'Source 2', 'Source 3', 'Source 4', 
                                    'Registration (Online)', 'Registration (Onspot)',
                                    'Registration (FLW&HCW)', 'Registration (18-45 Years)',
                                    'Registration (Above 45 Years)' ], axis = 1)

#changing the data type of elements of every column
for i in india_test_df.columns[2:]:
    india_test_df[i] = india_test_df[i].map(lambda x: 0 if(x=='' or x==' ') else x).fillna(0).map(
        lambda x: int(x.replace(',','')) if(isinstance(x,str)) else int(x))
    
#changing the date format and renaming the column
india_test_df['Tested As Of'] = india_test_df['Tested As Of'].map(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').strftime('%Y/%m/%d'))
india_test_df.rename(columns = {'Tested As Of' : 'Date'}, inplace = True)

#creating a new column and assigning values to it
india_test_df['Daily Vaccinated'] = 0

for i in range(0,len(india_test_df)):
    if(i==0):
        continue
    value = india_test_df.loc[i,'Total Individuals Vaccinated'] - india_test_df.loc[i-1,'Total Individuals Vaccinated']
    if(value==0):
        india_test_df.loc[i,'Daily Vaccinated'] = abs(india_test_df.loc[i-1,'Daily Vaccinated'])
        india_test_df.loc[i,'Total Individuals Vaccinated'] =  abs(india_test_df.loc[i-1,'Total Individuals Vaccinated'] + india_test_df.loc[i,'Daily Vaccinated'])
    else:
        india_test_df.loc[i,'Daily Vaccinated'] = value
        
india_test_df.to_excel(path+'india_test_df.xlsx', index=False)

State wise data

In [10]:
state_df = state_daily_df.groupby(by='State').max().reset_index()

#dropping Date column
del state_df['Date']

#changing data type of columns
for c in state_df.columns[2:]:
    state_df[c] = state_df[c].astype(int)
    
#calculating daily parameter values    
state_df['Daily Active'] = state_df['Daily Confirmed'] -state_df['Daily Deceased'] -state_df['Daily Recovered'] 
state_df['Total Active'] = state_df['Total Confirmed'] -state_df['Total Deceased'] -state_df['Total Recovered'] 
state_df = state_df.merge(state_vaccine_df.groupby(by='State').max(),on='State',how='outer')

#state wise population
state_df['Population'] = [417036,53903393,1570458,35607039,124799926,
                        1158473,29436231,615724,18710922,1586250,
                        63872399,28204692,7451955,13606320,38593948,
                        67562686,35699443,289023,73183,85358965,
                        123144223,3091545,3366710,1239244,2249695,
                        46356334,1413542,30141373,81032689,690251,
                        0,77841267,39362732,4169794,237882725,11250858,
                        99609303]

#sorting the data by confirmed cases
state_df = state_df.sort_values(by='Total Confirmed', ascending = False).reset_index()

#assigning rank 
state_df['Rank'] = list(range(1,len(state_df)+1))

#deleting unnecessary column
del state_df['index']

state_df.to_excel(path+'state_df.xlsx', index=False)

Hospitalization numbers data

In [11]:
hospital_df = pd.read_csv('hospital_df.csv')

#converting elements into integer
for c in hospital_df.columns[1:]:
    hospital_df[c] = hospital_df[c].astype(str).str.replace(',','').astype(int)
    
#sorting the dataframe
hospital_df = hospital_df.sort_values(by=' States/UTs')

#standardizing state names with other dataframes as data comes from different sources
keys = sorted(hospital_df[' States/UTs'].unique())
vals = sorted(state_df['State'].unique())
vals.insert(8,'Dadra and Nagar Haveli and Daman and Diu')
vals.remove('State Unassigned')
hospital_df[' States/UTs'] = hospital_df[' States/UTs'].replace(dict(zip(keys,vals)))
hospital_df = hospital_df.groupby(by=' States/UTs').sum()
hospital_df = hospital_df.reset_index()

hospital_df.to_excel(path+'hospital_df.xlsx', index=False)